# Query Library

In [1]:
from elasticsearch import Elasticsearch, helpers, exceptions
from elasticsearch.helpers import scan

In [2]:
import pandas as pd
import numpy as np
import os, fnmatch, datetime

In [3]:
# NLP Processing
# Import spaCy and load the language library
import spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
# Import the Matcher library
from spacy.matcher import Matcher


In [5]:
# Import the PhraseMatcher library
from spacy.matcher import PhraseMatcher


In [6]:
import re

# Query 

In [7]:
result_df = pd.DataFrame()
terms_searchsummary=pd.DataFrame()

In [8]:
indexname = "maxlibrary"
numberOfDocs = 1000

In [9]:
es =Elasticsearch(['http://localhost:9210'])

In [10]:
# Match all
response = es.search(
    index = indexname,
body = {
    "size": numberOfDocs,
    "_source":"_id",
    "query":{
        "match_all":{}
    }
})

GET maxlibrary/_search
{
  "_source": "Title", 
  "query": {
    "match":{
      "Content":{
        "query": "spirit jesus church ecclesia",
        "operator": "and"
      }}}}

In [11]:
searchquery='Spirit soul energy life chi'

In [12]:
# Match all
response = es.search(
    index = indexname,
body = {
    "size": numberOfDocs,
    "_source":"_id",
    "query":{
        "match":{
            "Content":{
                "query":searchquery,
                "operator":"and"
            }
        }
    }
})

In [13]:
def extract_context(text, search_words, width):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-13-8e71e7c4ff02>, line 2)

In [ ]:
titles=[]
contents = []
for doc in response['hits']['hits']:
    tempId = doc['_id']
    detailedResponse = es.search(
    index = indexname,
    body = {
        "size":numberOfDocs,
        "query":{
            "match":{
                "_id":tempId
            }
        }
    })
    entry=detailedResponse['hits']['hits'][0]
    title = entry['_source']['Title']
    titles.append(title)
    contents.append(entry['_source']['Content'])
    
    
    
    
print('''
    Searching for: %s
    
    Titles Matching Search - (%s):
    %s
    '''%(searchquery, len(titles),titles))
    

# NLP - Elasticsearch Initial search

In [ ]:
indexname = "maxlibrary"
numberOfDocs = 1000

In [106]:
es =Elasticsearch(['http://localhost:9210'])

In [107]:
# example doc 'BMGSzHkBNI5rijrm26YK'
tempId = 'BMGSzHkBNI5rijrm26YK'
detailedResponse = es.search(
    index = indexname,
    body = {
        "size":numberOfDocs,
        "query":{
            "match":{
                "_id":tempId
            }
        }
    })

In [108]:
entry=detailedResponse['hits']['hits'][0]
content = entry['_source']['Content']

In [109]:
content

'<h4>Header Page: 1</h4>\n\\AAANNAANAAAAANAAAAARANAAAAAANAAAAAAANAAAARAANAAARAAAAAARAAAAAAA Ae,\n\nEZ Lp\n\nSS SSS 0 RE RETR PORES\nEE AN) RSS\n\nROSICRUCIAN ORDER”\nA.M.O.R.C. *\n\nMASTER\nMONOGRAPH\n\nNEOPHYTE SECTION\n\nATRIUM 1 — INITIATION\n\n \n\x0c<h4>Footer Page: 1</h4>\n<h4>Header Page: 2</h4>\nROSICRUCIAN INITIATION\nATRIUM 1\n\n“Initiation brings into the realm of reason the purpose and into the\nrealm of emotion the spirit of one’s introduction into the Mysteries.”\n\nTHE STORY OF LIGHT\n\nAll the great avatars who gave to humanity spiritual doctrines or a\nmoral code enabling men and women to commune with the Cosmic\nand receive its [I}umination, were Torch Bearers of the Great Light\nwhich, thanks to them, lit the world through history’s darkest periods.\nWithout their help it is likely that the human race would not have\nadvanced to its present level of understanding and tolerance, even\nthough we must recognise that humanity still has far to go to be a\nmodel of Divine 

In [110]:
doc = nlp(content)

In [111]:
docx = doc

In [90]:
for sent in doc.sents:
    print(sent,'-----')

<h4>Header Page: 1</h4>
\AAANNAANAAAAANAAAAARANAAAAAANAAAAAAANAAAARAANAAARAAAAAARAAAAAAA Ae,

EZ Lp

SS SSS 0 RE RETR PORES
EE AN) RSS

ROSICRUCIAN ORDER”
A.M.O.R.C. *

MASTER
MONOGRAPH

 -----
NEOPHYTE SECTION

ATRIUM 1 — INITIATION

 
<h4>Footer Page: 1</h4>
<h4>Header Page: 2</h4>
 -----
ROSICRUCIAN INITIATION
ATRIUM 1

“Initiation brings into the realm of reason the purpose and into the
realm of emotion the spirit of one’s introduction into the Mysteries.” -----


THE STORY OF LIGHT -----


All the great avatars who gave to humanity spiritual doctrines or a
moral code enabling men and women to commune with the Cosmic
and receive its [I}umination, were Torch Bearers of the Great Light
which, thanks to them, lit the world through history’s darkest periods. -----

 -----
Without their help it is likely that the human race would not have
advanced to its present level of understanding and tolerance, even
though we must recognise that humanity still has far to go to be a
model of Divine

# matcher

In [18]:
matcher = Matcher(nlp.vocab)

In [19]:
pattern1 = [{'LOWER': 'divine'}]
pattern2 = [{'LOWER': 'divine'}, {'LOWER': 'perfection'}]
pattern3 = [{'LOWER': 'divine'}, {'IS_PUNCT': True}, {'LOWER': 'perfection'}]

# matcher.add('SolarPower', None, pattern1, pattern2, pattern3)
patterns = [pattern1]+[pattern2]+[pattern3]
matcher.add('SpiritLife', patterns)

In [20]:
found_matches = matcher(doc)
print(found_matches)

[(2626007918863180120, 203, 204), (2626007918863180120, 203, 205), (2626007918863180120, 2590, 2591), (2626007918863180120, 2601, 2602)]


In [21]:
for match_id, start, end in found_matches:
    string_id = nlp.vocab.strings[match_id]  # get string representation
    span = doc[start:end]                    # get the matched span
    print(match_id, string_id, start, end, span.text)

2626007918863180120 SpiritLife 203 204 Divine
2626007918863180120 SpiritLife 203 205 Divine Perfection
2626007918863180120 SpiritLife 2590 2591 Divine
2626007918863180120 SpiritLife 2601 2602 Divine


In [23]:
# Remove the old patterns to avoid duplication:
matcher.remove('SpiritLife')
#matcher.remove('SolarPower')

ValueError: [E175] Can't remove rule for unknown match pattern ID: SpiritLife

# phrasematcher

In [91]:
phrasematcher = PhraseMatcher(nlp.vocab, attr='LOWER')
# LOWER attribute makes the search case insensitive

In [92]:
# First, create a list of match phrases:
phrase_list = ['greater light', 'sign of the cross', 'evolution of humanity']

# Next, convert each phrase to a Doc object:
phrase_patterns = [nlp(text) for text in phrase_list]

# Pass each Doc object into matcher (note the use of the asterisk!):
phrasematcher.add('VoodooEconomics', None, *phrase_patterns)

# Build a list of matches:
matches = phrasematcher(doc)

In [70]:
# (match_id, start, end)
matches

[(3473369816841043438, 2302, 2304),
 (3473369816841043438, 2699, 2701),
 (3473369816841043438, 3987, 3991),
 (3473369816841043438, 4000, 4004),
 (3473369816841043438, 4024, 4026),
 (3473369816841043438, 4324, 4327)]

In [27]:
phrasematcher.remove('VoodooEconomics')

In [28]:
# Build a list of sentences
sents = [sent for sent in doc.sents]

# In the next section we'll see that sentences contain start and end token values:
print(sents[0].start, sents[0].end)

0 39


In [29]:
doc[4310:4340]

often difficult, when considering the founders of spiritual
doctrines which marked the evolution of humanity, to make a
distinction between the historical truths and legendary aspects

In [30]:
# Iterate over the sentence list until the sentence end value exceeds a match start value:
for sent in sents:
    for m in matches:
        if (m[1] < sent.end) & (m[1]>= sent.start):  # this is the fifth match, that starts at doc3[673]
            print('\n[%s]-[%s]:-\n%s\n'%(sent.start, sent.end, sent))
            break


[2274]-[2324]:-


Light the candle on your left and, immediately after
having done so, say: “May this Sacred Light,
symbol of the Greater Light of Cosmic Wisdom,
cast its radiance in the midst of darkness and
illuminate my path.”


[2682]-[2712]:-
Of
that you must always be conscious, for as long as
you seek the Greater Light, your inner light will
guide your steps.




[3976]-[4022]:-
Then close
this special initiation by making the Rosicrucian
Sign of the Cross and saying:

‘“‘Before the Sign of the Cross and in the presence
of the Guardian of this Sanctum, I close this
ceremony.


[4022]-[4052]:-
May the Greater Light of Cosmic
Wisdom enlighten me forever, so that I may be
worthy of the Knowledge which will be entrusted
to me.


[4308]-[4354]:-
It is often difficult, when considering the founders of spiritual
doctrines which marked the evolution of humanity, to make a
distinction between the historical truths and legendary aspects of
their lives, for such beings are always describe

In [32]:
sents[0]

<h4>Header Page: 1</h4>
\AAANNAANAAAAANAAAAARANAAAAAANAAAAAAANAAAARAANAAARAAAAAARAAAAAAA Ae,

EZ Lp

SS SSS 0 RE RETR PORES
EE AN) RSS

ROSICRUCIAN ORDER”
A.M.O.R.C. *

MASTER
MONOGRAPH


# phrasematcher - module

> Implementation of a generic phrase matching

* set list of phrases
* from Elasticsearch list of all documents with matches -> matching_population
* for all documents in matching_population
    * create phrasematcher
    * extract all the sentences

In [14]:
phrasematchername = 'search'

In [15]:
# First, create a list of match phrases:
phrase_list = ['greater light', 'sign of the cross', 'evolution of humanity']

In [16]:
indexname = "maxlibrary"
numberOfDocs = 1000

In [17]:
es =Elasticsearch(['http://localhost:9210'])

In [20]:
doc_ids =[]
for phrase in phrase_list:
    print('Processing "%s"'%(phrase))
    detailedResponse = es.search(
    index = indexname,
    body = {
        "size":numberOfDocs,
        "query":{
            "match_phrase":{
                "Content":{
                    "query": phrase
                }
            }
        }
    })

    entries=detailedResponse['hits']['hits']
    print(' -> (%s)'%(len(entries)))
    for entry in entries:
        doc_ids.append(entry['_id'])

Processing "greater light"
 -> (7)
Processing "sign of the cross"
 -> (6)
Processing "evolution of humanity"
 -> (6)


In [21]:
doc_ids=list(set(sorted(doc_ids)))

In [22]:
print(len(doc_ids))

13


In [23]:
doc_ids # contains the list of documents with at least a match

['NMEczXkBNI5rijrmvKav',
 'NcEkzXkBNI5rijrmXKbk',
 'M8EKzXkBNI5rijrmg6YE',
 'P8GozXkBNI5rijrmhaYF',
 'KcHMzHkBNI5rijrmQKak',
 'PMGGzXkBNI5rijrm16YM',
 '_cFnzHkBNI5rijrm3aXO',
 'PsGazXkBNI5rijrmdabE',
 'J8GlzHkBNI5rijrm-qZc',
 'BMGSzHkBNI5rijrm26YK',
 'PcGLzXkBNI5rijrmRqZO',
 'QMGxzXkBNI5rijrmk6Zh',
 'F8GdzHkBNI5rijrmQqY-']

In [24]:
nlp.max_length = 3000000

In [25]:
# Build a list of matches for all the documents
for doc_id in doc_ids:
    print('Processing document id: "%s"\nIndex: %s'%(doc_id, indexname))
    detailedResponse = es.search(
    index = indexname,
    body = {
        "size":numberOfDocs,
        "query":{
            "match":{
                "_id":doc_id
            }
        }
    })
    
    entry=detailedResponse['hits']['hits'][0]
    content = entry['_source']['Content']
    title = entry['_source']['Title']
    
    doc = nlp(content)
   # doc = docx
    #print(doc)
    
    # Next, convert each phrase to a Doc object:
    phrase_patterns = [nlp(text) for text in phrase_list]
    print('Phrase Patterns: %s'%(phrase_patterns))
    phrasematcher = PhraseMatcher(nlp.vocab, attr='LOWER')
    # LOWER attribute makes the search case insensitive

    # Pass each Doc object into matcher (note the use of the asterisk!):
    phrasematcher.add(phrasematchername, None, *phrase_patterns)
    
    # Build a list of matches:
    matches = phrasematcher(doc)
    print('Number of matches: %s'%(len(matches)))
    
    # Build a list of sentences
    sents = [sent for sent in doc.sents]
    
    print('\nTitle: %s'%(title))
    # Iterate over the sentence list until the sentence end value exceeds a match start value:
    prev_sent = ''
    for sent in sents:
        for m in matches:
            if (m[1] < sent.end) & (m[1]>= sent.start):  # this is the fifth match, that starts at doc3[673]
                print('\nDocument Location:[%s]-[%s]:-\n%s\n ->%s<-\n%s\n'%(sent.start, sent.end, 
                                sents[sents.index(sent)-1],sent,sents[sents.index(sent)+1]))
                break

Processing document id: "NMEczXkBNI5rijrmvKav"
Index: maxlibrary
Phrase Patterns: [greater light, sign of the cross, evolution of humanity]
Number of matches: 10

Title: Rosicrucian Manual AMORC.pdf

Document Location:[14053]-[14097]:-


When the Sanctum is entered for any legitimate purpose, he who
enters it may pass only to the center of it, and must leave it from the
same point by which he entered it.
 ->Likewise must each who enters
it immediately face the East, from the center of it, and make the
“Sign of the Cross” before doing that act or performing that function
which necessitated entrance into the Sanctum.<-
In leaving the Sanc-
tum, he who entered it must only leave by the same point of en-
trance.


Document Location:[15214]-[15238]:-
Guardian

 
<h4>Footer Page: 24</h4>
<h4>Header Page: 25</h4>

 ->THE SIGN OF THE CROSS

Reference has been made many times in the preceding pages to
the Sign of the Cross.<-
An explanation of this term and Sign is
necessary.


Document Locati

Phrase Patterns: [greater light, sign of the cross, evolution of humanity]
Number of matches: 1

Title: A - Positio RC .pdf

Document Location:[6035]-[6061]:-
Furthermore, people may be estranged because the religions have
alienated themselves from spirituality.
 ->But spirituality, although SR+C\
immutable in essence, constantly seeks to express itself through

channels increasingly suited to the evolution of humanity.<-
\\

--14--
<h4>Footer Page: 14</h4>
<h4>Header Page: 15</h4>


Processing document id: "PsGazXkBNI5rijrmdabE"
Index: maxlibrary
Phrase Patterns: [greater light, sign of the cross, evolution of humanity]
Number of matches: 1

Title: The Rosicrucians Their Rites and Mysteries.pdf

Document Location:[17048]-[17108]:-
THE PHILOSOPHER HUME.
 ->39

that no more truth is vouchsafed to man than he knows
how to use: most of his uses, even of his little quantum of
truth, being perverted, He must await other states for
greater light, and to become a higher creature—should that


In [47]:
print(len(doc))

4501


In [48]:
print(nlp.max_length)

1000000
